In [ ]:
# default_exp core

# Ocetrac

> Track and label marine heatwaves from geospatial data. 

In [ ]:
#export
from nbdev.showdoc import *
import xarray as xr
import numpy as np
import scipy.ndimage
from skimage.measure import label, regionprops 
# import matplotlib.pyplot as plt 


## Functions

In [ ]:
#export
def _morphological_operations(da, radius=8): 
    '''Converts xarray.DataArray to binary, defines structuring element, and performs morphological closing then opening.
    Parameters
    ----------
    da     : xarray.DataArray
            The data to label
    radius : int
            Length of grid spacing to define the radius of the structing element used in morphological closing and opening.
        
    '''
    
    # Convert images to binary. All positive values == 1, otherwise == 0
    bitmap_binary = da.where(da>0, drop=False, other=0)
    bitmap_binary = bitmap_binary.where(bitmap_binary==0, drop=False, other=1)
    
    # Define structuring element
    diameter = radius*2
    x = np.arange(-radius, radius+1)
    x, y = np.meshgrid(x, x)
    r = x**2+y**2 
    se = r<radius**2

    def binary_open_close(bitmap_binary):
        bitmap_binary_padded = np.pad(bitmap_binary,
                                      ((diameter, diameter), (diameter, diameter)),
                                      mode='wrap')
        s1 = scipy.ndimage.binary_closing(bitmap_binary_padded, se, iterations=1)
        s2 = scipy.ndimage.binary_opening(s1, se, iterations=1)
        unpadded= s2[diameter:-diameter, diameter:-diameter]
        return unpadded
    
    mo_binary = xr.apply_ufunc(binary_open_close, bitmap_binary,
                               input_core_dims=[['lat', 'lon']],
                               output_core_dims=[['lat', 'lon']],
                               output_dtypes=[bitmap_binary.dtype],
                               vectorize=True,
                               dask='parallelized')
    
    return mo_binary


In [ ]:
#hide
################## TEST ##################

# Let's create a feature image set with 3 time steps and containing 5 different Gaussian blobs to test the morphological operations. 
# Three of these blobs will be spatially offset at each time step, but will still overlap. Two other blobs will appear at time step 3. 

# Create blobs
lon = np.arange(0, 360) + 0.5
lat = np.arange(-90, 90) + 0.5
x, y = np.meshgrid(lon, lat)

x0 = 180; x1 = 225; x2 = 360; x3 = 1; x4 = 80
y0 = 0; y1 = 20; y2 = -50; y4 = 40
sigma0 = 15; sigma1 = 25; sigma2 = 30; sigma4 = 10

blob0 = np.exp(-((x - x0)**2 + (y - y0)**2)/(2*sigma0**2))
blob1 = np.exp(-((x - x1)**2 + (y - y1)**2)/(2*sigma1**2))
blob2 = np.exp(-((x - x2)**2 + (y - y2)**2)/(2*sigma2**2))
blob3 = np.exp(-((x - x4)**2 + (y - y4)**2)/(2*sigma4**2))
blob4 = np.exp(-((x - x3)**2 + (y - y2)**2)/(2*sigma2**2))
blob5 = np.exp(-((x - x2)**2 + (y - y4)**2)/(2*sigma0**2))
blob6 = np.exp(-((x - x3)**2 + (y - y4)**2)/(2*sigma4**2))

da = xr.DataArray((blob0+blob1+blob3)-.5, dims=['lat', 'lon'],
                  coords={'lat': lat, 'lon': lon})
da_shift_01 = da.shift(lon=0, lat=-20, fill_value=-.5)
da_shift_02 = da.shift(lon=0, lat=-40, fill_value=-.5)+(blob2+blob4+blob5+blob6)
da_shift_03 = da.shift(lon=0, lat=-40, fill_value=-.5)+(blob2+blob5+blob6)


da_3D = xr.concat((da.expand_dims(dim='time'),
                   da_shift_01.expand_dims(dim='time'),
                   da_shift_02.expand_dims(dim='time'),
                   da_shift_03.expand_dims(dim='time'),), dim='time')


# ##### A useful plot:
# da_3D[0,:,:].plot(vmin=-0.6, vmax=0.6, cmap='RdBu_r', extend='both')
# c1 = da_3D[0,:,:].plot.contour(levels=[0], colors='k', linewidths=2, linestyles=':')
# c2 = da_3D[1,:,:].plot.contour(levels=[0], colors='k', linewidths=2, linestyles='--')
# c3 = da_3D[2,:,:].plot.contour(levels=[0], colors='k', linewidths=2, linestyles='-')
# plt.arrow(80,40,0,-38, head_width=8, head_length=6, lw=2, color='magenta', zorder=4)
# plt.arrow(210,15,0,-38, head_width=8, head_length=6, lw=2, color='magenta', zorder=4)
# h1,_ = c1.legend_elements(); h2,_ = c2.legend_elements(); h3,_ = c3.legend_elements()
# plt.legend([h1[0], h2[0], h3[0]], ['t=0', 't=1', 't=2'])
# plt.title('Feature Field at t=0 \n Best Guess Objects Contoured at t=[0,1,2]', fontsize=14);


In [ ]:
#hide 
################## TEST ##################

# Now we will use Ocetrac to detect objects. 
# You can see, in this example, our best guess (shown previously) is pretty similar to Ocetrac (below)! 
# In the real world, these features are never perfectly Gaussian.

# Find edges of blobs using morphological image processing.
da_3D_dask = da_3D.chunk({'time': 1})
mo_binary = _morphological_operations(da_3D_dask, radius=8)
import dask.array as dsa
assert isinstance(mo_binary.data, dsa.Array)

In [ ]:
#hide
################## TEST ##################

# Now we can assert that Ocetrac and our best guess estimates overlap by at least 80% and that the sum of the Ocetrac detected pixels equals 17411.
ocetrac_blobs = da_3D.where(mo_binary==True, drop=False, other=np.nan) 
best_guess_blobs = da_3D.where(da_3D>0, drop=False, other=np.nan) 

part = ocetrac_blobs.isin(best_guess_blobs)
whole = best_guess_blobs.isin(best_guess_blobs)
assert part.sum().values/whole.sum().values*100 >= 80
assert part.sum().values == 26122

In [ ]:
#export
def _id(binary_images):
    '''label object from binary images, without trackin in time. '''
    
    unique_labels, num = xr.apply_ufunc(
        label, 
        binary_images,
        kwargs={'return_num': True, 'connectivity': 2},
        input_core_dims=[['lat', 'lon', ]],
        output_core_dims=[['lat', 'lon'], []],
        output_dtypes=['i4', 'i4'],
        dask='parallelized',
        vectorize=True
    )

    #non_core_dims = set(binary_images.dims) - {'lat', 'lon'}
    # TODO: stop assuming 3D images
    
    offset = num.cumsum().shift(time=1, fill_value=0)
    unique_labels = xr.where(unique_labels > 0, unique_labels + offset, 0)
    
    return unique_labels

In [ ]:
#hide
################## TEST ##################

# Using the test data, let's label the objects we've identified. We should have 4 uniquely labeled blobs.
ID = _id(mo_binary)
assert isinstance(ID.data, dsa.Array)
ID.load()
assert ID[2,:,:].max() == 10.
assert all([i in ID[2,:,:] for i in range(5,11)])

In [ ]:
def _filter_area(mo_binary, min_size_quartile):
    '''calculatre area with regionprops'''
    
    unique_labels = _id(mo_binary)
    props = regionprops(unique_labels.values.astype('int'))

    labelprops = [p.label for p in props]
    labelprops = xr.DataArray(labelprops, dims=['label'], coords={'label': labelprops}) 
    coords = [p.coords for p in props] # time, lat, lon

    area = []
    res = mo_binary.lat[1].values-mo_binary.lat[0].values # resolution of latitude
    for i in range(len(coords)):  
        area.append(np.sum((res*111)*np.cos(np.radians(mo_binary.lat[coords[i][:,0]].values)) * (res*111)))
    area = xr.DataArray(area, dims=['label'], coords={'label': labelprops})  
    min_area = np.percentile(area, min_size_quartile*100)
    print('min area (km2) \t', min_area)  
    
    keep_labels = labelprops.where(area>=min_area, drop=True)
    ID_area = xr.DataArray(np.isin(unique_labels, keep_labels).reshape(unique_labels.shape),
                               dims=unique_labels.dims, coords=unique_labels.coords)

    return area, min_area, ID_area, labelprops

In [ ]:
#hide
################## TEST [_filter_area] ##################

# Given the blobs detected by Ocetrac, what is the minimum area defined by computing the 75th percentile of the blob area distribution.
# Given this size criteria, only blob #2 is considered by Ocetrac.

# id_wrap = id_3D_wrap.where(id_3D_wrap!= 0, drop=False, other=np.nan)
min_size_quartile = .75

area, min_area, ID_area, labelprops = _filter_area(mo_binary, min_size_quartile)
keep_labels = labelprops.where(area>=min_area, drop=True)
assert (area.label[area>=min_area] == keep_labels).all()

tot_area = int(np.sum(area.values))
small_area = area.where(area<=min_area, drop=True)
small_area = int(np.sum(small_area.values))
percent_area_kept = 1-(small_area/tot_area)
print('percent of total area kept = {}'.format(np.round(percent_area_kept*100,2)))
# Is the percent area kept less than or equal to the minimum size quantile?
assert percent_area_kept <= min_size_quartile


In [ ]:
#export
from skimage.measure import label as label_np

def _label_either(data, **kwargs):
    if isinstance(data, dsa.Array):
        try:
            from dask_image.ndmeasure import label as label_dask
            def label_func(a, **kwargs):
                ids, num = label_dask(a, **kwargs)
                return ids
        except ImportError:
            raise ImportError(
                "Dask_image is required to use this function on Dask arrays. "
                "Either install dask_image or else call .load() on your data."
            )
    else:
        label_func = label_np
    return label_func(data, **kwargs)


In [ ]:
def _wrap(labels):
    ''' Impose periodic boundary and wrap labels'''
    first_column = labels[..., 0]
    last_column = labels[..., -1]
    
    # Here's a test to find the rows where both column contain an integer greater than 0
    overlap = first_column * last_column 
    match = np.where(overlap > 0) # We get a match where the product of the two columns is > 0
    
    # Where our test passes, find the unique labels in the last column that we want to replace.
    bad_label = np.unique(last_column[match[0],match[1]])
    
    # Similarly, find the unique labels of the frist column that we will use to relabel the last_column.
    new_label = np.unique(first_column[match[0],match[1]])
    
    # This loop iterates over the bad labels and replaces all instances of this bad label with the new label.
    for i in enumerate(bad_label):
        labels[np.where(labels == i[1])] = new_label[i[0]]
    
    # Reorder labels
    new_labels = xr.DataArray(np.unique(labels, return_inverse=True)[1].reshape(labels.shape))
    
    # recalculate the total number of labels 
    N = np.max(new_labels)

    return new_labels, N


In [ ]:
#export
def track(da, radius=8, area_quantile=0.75):
    '''Image labeling and tracking.
    
    Parameters
    ----------
    da : xarray.DataArray
        The data to label.
    
    radius : int
        size of the structuring element used in morphological opening and closing.
        
    area_quantile : float
        quantile used to define the threshold of the smallest area object retained in tracking.
        
    Returns
    -------
    labels : xarray.DataArray
        Integer labels of the connected regions.
    '''
        
    # Converts data to binary, defines structuring element, and performs morphological closing then opening
    binary_images = _morphological_operations(da, radius=radius) 
    
    area, min_area, ID_area, labelprops = _filter_area(mo_binary, 0)

    labels, num = _label_either(ID_area, return_num= True, connectivity=3)

    new_labels, N = _wrap(labels)


    ### ! Reapply land maks HERE

    # Calculate Percent of total MHW area retained
    tot_area = int(np.sum(area.values))
    small_area = area.where(area<=min_area, drop=True)
    small_area = int(np.sum(small_area.values))
    percent_area_kept = 1-(small_area/tot_area)

    features = _id(ID_area)
    features = features.rename('labels')
    features.attrs['min_area'] = min_area
    features.attrs['percent_area_kept'] = percent_area_kept
    print('inital features identified \t', int(features.max().values))
    

    print('final features tracked \t', int(N))
    
    return new_labels, N

In [ ]:
new_labels, N = track(da_3D_dask, radius=8, area_quantile=0.75)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
# this is the same as running nbdev_build_lib